In [6]:
import numpy as np
import pandas as pd
import xgboost as xgb
import pyarrow as pa
import pyarrow.parquet as pq
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import functions as fc
import matplotlib.pyplot as plt

## comparing 0.005 GeV models

In [7]:
get_models = ["over0_150.txt","03_model500.txt","04_model500.txt","all_model500.txt"]
labels = ["5.0 MeV with n = 150", "(0.005 + 3.0) GeV","(0.005 + 4.6) GeV", "(0.005 + 3.0 + 4.6) GeV"  ]
legends = ["n=150", "5 MeV + 3.0 GeV", "5.0 GeV + 4.6 GeV","all masses"]
legend_location = [2,2,1,1,1,1,1]

df = pq.read_table("/ceph/aavocone/Datasets/0_large.parquet")
df = df.to_pandas()


X = df[df.columns[:-1]]    #exclude "signal efficiency [%]" "classification" "B_sig_isSignalAcceptMissingNeutrino"
y = df["signal"]   
xtrain,xtest,ytrain,ytest = train_test_split(X, y, test_size = 0.33, stratify = y)
xtrain,xval,ytrain,yval = train_test_split(xtrain, ytrain, test_size = 0.5)
clf = XGBClassifier()

bckgrnd = 1-ytest
print("Number of total background events:",sum(bckgrnd))

sig_eff = []
bg_eff = []
pfom_list = []
maximum = []

for index, models in enumerate(get_models):
    clf.load_model(f"/ceph/aavocone/PFOM_models/{models}")
    yprob = clf.predict_proba(xtest)[:,1]
    sh,bh,s,b,bin_edges = fc.efficiency(yprob,ytest, eff_type=0)
    pfom = fc.PFOM(s,b,bh,5)
    pfom_list.append(pfom)
    sig_eff.append(s)
    bg_eff.append(b)
    max_index = np.where(pfom==max(pfom))[0][0] #np.where = tuple(array[index_value]), to get index you need to specify location in tuple and in array... 
    max_value = pfom[max_index]
    maximum.append([max_index/100,max_value,s[max_index],1-b[max_index]])

    print("Model:",legends[index])
    print(f"Cut at xgb probability {max_index/100}")
    b_max = bh >= max_index/100
    b_max = b_max[b_max!=0]
    print("Number of background events after cut:",len(b_max))
    print()

"""
# signal efficiency
plt.figure(figsize=(9,6))
plt.plot(bin_edges, 100*(sig_eff[0]), label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0], 100*maximum[0][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[1]), label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0], 100*maximum[1][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[2]), label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0], 100*maximum[2][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[3]), label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0], 100*maximum[3][2], marker = "x")
plt.title("data sample = 0.005 GeV",fontsize=15)
plt.ylabel("signal efficiency [%]",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/0/signal0.pdf", format="pdf",bbox_inches="tight")
plt.show()

#background rejection
plt.figure(figsize=(9,6))
plt.plot(bin_edges, 100*(1-bg_eff[0]), label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0], 100*maximum[0][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[1]), label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0], 100*maximum[1][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[2]), label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0], 100*maximum[2][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[3]), label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0], 100*maximum[3][3], marker = "x")
plt.title("data sample = 0.005 GeV",fontsize=15)
plt.ylabel("background rejection [%]",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/0/background0.pdf", format="pdf",bbox_inches="tight")
plt.show()



#PFOM----------------
plt.figure(figsize=(9,6))
plt.plot(bin_edges, pfom_list[0], label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0],maximum[0][1], marker = "x")

plt.plot(bin_edges, pfom_list[1], label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0],maximum[1][1], marker = "x")

plt.plot(bin_edges, pfom_list[2], label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0],maximum[2][1], marker = "x")

plt.plot(bin_edges, pfom_list[3], label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0],maximum[3][1], marker = "x")
plt.title("Punzi figure of merit comparison",fontsize=15)
plt.ylabel("Punzi FOM",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/0/PFOM0.pdf", format="pdf",bbox_inches="tight")
plt.show()
"""

Number of total background events: 45875373.0
Model: n=150
Cut at xgb probability 0.98
Number of background events after cut: 119383

Model: 5 MeV + 3.0 GeV
Cut at xgb probability 0.96
Number of background events after cut: 144073

Model: 5.0 GeV + 4.6 GeV
Cut at xgb probability 0.95
Number of background events after cut: 235678

Model: all masses
Cut at xgb probability 0.92
Number of background events after cut: 502348



'\n# signal efficiency\nplt.figure(figsize=(9,6))\nplt.figure(figsize=(9,6))\nplt.plot(bin_edges, 100*(sig_eff[0]), label = legends[0], alpha = 0.7)\nplt.scatter(maximum[0][0], 100*maximum[0][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[1]), label = legends[1], alpha = 0.7)\nplt.scatter(maximum[1][0], 100*maximum[1][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[2]), label = legends[2], alpha = 0.7)\nplt.scatter(maximum[2][0], 100*maximum[2][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[3]), label = legends[3], alpha = 0.7)\nplt.scatter(maximum[3][0], 100*maximum[3][2], marker = "x")\nplt.title("data sample = 0.005 GeV",fontsize=15)\nplt.ylabel("signal efficiency [%]",fontsize=15)\nplt.xlabel("xgb probability", fontsize = 20)\nplt.legend(framealpha = 0)\nplt.savefig(f"/work/aavocone/week12/0/signal0.pdf", format="pdf",bbox_inches="tight")\nplt.show()\n\n#background rejection\nplt.figure(figsize=(9,6))\nplt.plot(bin_edges, 100*(1-bg_eff[0]), label = legends[0], alpha =

In [8]:
print("          Model          :      [xgb, max punzi] ")
print("-------------------------------------------------------------")
for i in range(len(labels)):
    print("{0:25s}: {1:}".format(labels[i],maximum[i]))

          Model          :      [xgb, max punzi] 
-------------------------------------------------------------
5.0 MeV with n = 150     : [0.98, 0.00032783932957298497, 0.01849, 0.997397666935591]
(0.005 + 3.0) GeV        : [0.96, 0.0001678055776323262, 0.01288, 0.9968594696766825]
(0.005 + 4.6) GeV        : [0.95, 0.00013302195897099483, 0.01216, 0.9948626466753742]
(0.005 + 3.0 + 4.6) GeV  : [0.92, 7.202683713088836e-05, 0.01352, 0.9890497239117816]


## comparing 3.0 GeV models

In [10]:
get_models = ["3_0_500.txt","03_model500.txt","34_model500.txt","all_model500.txt"]
labels = ["3.0 GeV with n = 500", "(0.005 + 3.0) GeV","(3.0 + 4.6) GeV", "(0.005 + 3.0 + 4.6) GeV"  ]
legends = ["n=500","5 MeV + 3.0 GeV", "3.0 GeV + 4.6 GeV","all masses"]
legend_location = [2,1,2,2,2,2]
df = pq.read_table("/ceph/aavocone/Datasets/3_large.parquet")
df = df.to_pandas()

X = df[df.columns[:-1]]    #exclude "signal efficiency [%]" "classification" "B_sig_isSignalAcceptMissingNeutrino"
y = df["signal"]   
xtrain,xtest,ytrain,ytest = train_test_split(X, y, test_size = 0.33, stratify = y)
xtrain,xval,ytrain,yval = train_test_split(xtrain, ytrain, test_size = 0.5)
clf = XGBClassifier()

bckgrnd = 1-ytest
print("Number of total background events:",sum(bckgrnd))

sig_eff = []
bg_eff = []
pfom_list = []
maximum = []

for index, models in enumerate(get_models):
    clf.load_model(f"/ceph/aavocone/PFOM_models/{models}")
    yprob = clf.predict_proba(xtest)[:,1]
    sh,bh,s,b,bin_edges = fc.efficiency(yprob,ytest, eff_type=0)
    pfom = fc.PFOM(s,b,bh,5)
    pfom_list.append(pfom)
    sig_eff.append(s)
    bg_eff.append(b)
    max_index = np.where(pfom==max(pfom))[0][0] #np.where = tuple(array[index_value]), to get index you need to specify location in tuple and in array... 
    max_value = pfom[max_index]
    maximum.append([max_index/100,max_value,s[max_index],1-b[max_index]])
    
    print("Model:",legends[index])
    print(f"Cut at xgb probability {max_index/100}")
    b_max = bh >= max_index/100
    b_max = b_max[b_max!=0]
    print("Number of background events after cut:",len(b_max))
    print()


""" 
# signal efficiency
plt.figure(figsize=(9,6))
plt.figure(figsize=(9,6))
plt.plot(bin_edges, 100*(sig_eff[0]), label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0], 100*maximum[0][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[1]), label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0], 100*maximum[1][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[2]), label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0], 100*maximum[2][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[3]), label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0], 100*maximum[3][2], marker = "x")
plt.title("data sample = 3.0 GeV",fontsize=15)
plt.ylabel("signal efficiency [%]",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/3/signal3.pdf", format="pdf",bbox_inches="tight")
plt.show()

#background rejection
plt.figure(figsize=(9,6))
plt.plot(bin_edges, 100*(1-bg_eff[0]), label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0], 100*maximum[0][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[1]), label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0], 100*maximum[1][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[2]), label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0], 100*maximum[2][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[3]), label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0], 100*maximum[3][3], marker = "x")
plt.title("data sample = 3.0 GeV",fontsize=15)
plt.ylabel("background rejection [%]",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/3/background3.pdf", format="pdf",bbox_inches="tight")
plt.show()






plt.figure(figsize=(9,6))
plt.plot(bin_edges, pfom_list[0], label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0],maximum[0][1], marker = "x")

plt.plot(bin_edges, pfom_list[1], label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0],maximum[1][1], marker = "x")

plt.plot(bin_edges, pfom_list[2], label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0],maximum[2][1], marker = "x")

plt.plot(bin_edges, pfom_list[3], label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0],maximum[3][1], marker = "x")


plt.title("Punzi figure of merit comparison",fontsize=15)
plt.ylabel("Punzi FOM",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(fontsize = 15 , framealpha = 0)
plt.savefig(f"/work/aavocone/week12/3/PFOM3.pdf", format="pdf",bbox_inches="tight")
plt.show()
"""



Number of total background events: 45875372.0
Model: n=500
Cut at xgb probability 0.95
Number of background events after cut: 122616

Model: 5 MeV + 3.0 GeV
Cut at xgb probability 0.92
Number of background events after cut: 309416

Model: 3.0 GeV + 4.6 GeV
Cut at xgb probability 0.82
Number of background events after cut: 995027

Model: all masses
Cut at xgb probability 0.81
Number of background events after cut: 1186003



'\n# signal efficiency\nplt.figure(figsize=(9,6))\nplt.figure(figsize=(9,6))\nplt.plot(bin_edges, 100*(sig_eff[0]), label = legends[0], alpha = 0.7)\nplt.scatter(maximum[0][0], 100*maximum[0][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[1]), label = legends[1], alpha = 0.7)\nplt.scatter(maximum[1][0], 100*maximum[1][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[2]), label = legends[2], alpha = 0.7)\nplt.scatter(maximum[2][0], 100*maximum[2][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[3]), label = legends[3], alpha = 0.7)\nplt.scatter(maximum[3][0], 100*maximum[3][2], marker = "x")\nplt.title("data sample = 3.0 GeV",fontsize=15)\nplt.ylabel("signal efficiency [%]",fontsize=15)\nplt.xlabel("xgb probability", fontsize = 20)\nplt.legend(framealpha = 0)\nplt.savefig(f"/work/aavocone/week12/3/signal3.pdf", format="pdf",bbox_inches="tight")\nplt.show()\n\n#background rejection\nplt.figure(figsize=(9,6))\nplt.plot(bin_edges, 100*(1-bg_eff[0]), label = legends[0], alpha = 0

##

In [11]:
print("          Model          :      [xgb, max punzi] ")
print("-------------------------------------------------------------")
for i in range(len(labels)):
    print("{0:25s}: {1:}".format(labels[i],maximum[i]))

          Model          :      [xgb, max punzi] 
-------------------------------------------------------------
3.0 GeV with n = 500     : [0.95, 0.0004486135061611418, 0.02732, 0.9973271933358927]
(0.005 + 3.0) GeV        : [0.92, 0.00028789539953780104, 0.03204, 0.9932552917500048]
(3.0 + 4.6) GeV          : [0.82, 0.00015959240644084337, 0.03886, 0.9783102140294361]
(0.005 + 3.0 + 4.6) GeV  : [0.81, 0.00013620976221420214, 0.0391, 0.9741472832089514]


## comparing 4.6 GeV models

In [12]:
get_models = ["new_4_6_model500.txt","04_model500.txt","34_model500.txt","all_model500.txt"]
labels = ["4.6 GeV with n = 500", "(0.005 + 4.6) GeV","(3.0 + 4.6) GeV", "(0.005 + 3.0 + 4.6) GeV"]
legends = ["n=500","5 MeV + 4.6 GeV", "3.0 GeV + 4.6 GeV","all masses"]
df = pq.read_table("/ceph/aavocone/Datasets/4_large.parquet")
df = df.to_pandas()

X = df[df.columns[:-1]]    #exclude "signal efficiency [%]" "classification" "B_sig_isSignalAcceptMissingNeutrino"
y = df["signal"]   
xtrain,xtest,ytrain,ytest = train_test_split(X, y, test_size = 0.33, stratify = y)
xtrain,xval,ytrain,yval = train_test_split(xtrain, ytrain, test_size = 0.5)
clf = XGBClassifier()
bckgrnd = 1-ytest
print("Number of total background events:",sum(bckgrnd))

sig_eff = []
bg_eff = []
pfom_list = []
maximum = []

for index, models in enumerate(get_models):
    clf.load_model(f"/ceph/aavocone/PFOM_models/{models}")
    yprob = clf.predict_proba(xtest)[:,1]
    sh,bh,s,b,bin_edges = fc.efficiency(yprob,ytest, eff_type=0)
    pfom = fc.PFOM(s,b,bh,5)
    pfom_list.append(pfom)
    sig_eff.append(s)
    bg_eff.append(b)
    max_index = np.where(pfom==max(pfom))[0][0] #np.where = tuple(array[index_value]), to get index you need to specify location in tuple and in array... 
    max_value = pfom[max_index]
    maximum.append([max_index/100,max_value,s[max_index],1-b[max_index]])

    print("Model:",legends[index])
    print(f"Cut at xgb probability {max_index/100}")
    b_max = bh >= max_index/100
    b_max = b_max[b_max!=0]
    print("Number of background events after cut:",len(b_max))
    print()



"""
# signal efficiency
plt.figure(figsize=(9,6))
plt.plot(bin_edges, 100*(sig_eff[0]), label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0], 100*maximum[0][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[1]), label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0], 100*maximum[1][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[2]), label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0], 100*maximum[2][2], marker = "x")
plt.plot(bin_edges, 100*(sig_eff[3]), label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0], 100*maximum[3][2], marker = "x")
plt.title("data sample = 4.6 GeV",fontsize=15)
plt.ylabel("signal efficiency [%]",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/4/signal4.pdf", format="pdf",bbox_inches="tight")
plt.show()

#background rejection
plt.figure(figsize=(9,6))
plt.plot(bin_edges, 100*(1-bg_eff[0]), label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0], 100*maximum[0][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[1]), label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0], 100*maximum[1][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[2]), label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0], 100*maximum[2][3], marker = "x")
plt.plot(bin_edges, 100*(1-bg_eff[3]), label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0], 100*maximum[3][3], marker = "x")
plt.title("data sample = 4.6 GeV",fontsize=15)
plt.ylabel("background rejection [%]",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/4/background4.pdf", format="pdf",bbox_inches="tight")
plt.show()




plt.figure(figsize=(9,6))
plt.plot(bin_edges, pfom_list[0], label = legends[0], alpha = 0.7)
plt.scatter(maximum[0][0],maximum[0][1], marker = "x")

plt.plot(bin_edges, pfom_list[1], label = legends[1], alpha = 0.7)
plt.scatter(maximum[1][0],maximum[1][1], marker = "x")

plt.plot(bin_edges, pfom_list[2], label = legends[2], alpha = 0.7)
plt.scatter(maximum[2][0],maximum[2][1], marker = "x")

plt.plot(bin_edges, pfom_list[3], label = legends[3], alpha = 0.7)
plt.scatter(maximum[3][0],maximum[3][1], marker = "x")

plt.title("Punzi figure of merit comparison",fontsize=15)
plt.ylabel("Punzi FOM",fontsize=15)
plt.xlabel("xgb probability", fontsize = 20)
plt.legend(framealpha = 0)
plt.savefig(f"/work/aavocone/week12/4/PFOM4.pdf", format="pdf",bbox_inches="tight")
plt.show()
"""

Number of total background events: 45875372.0
Model: n=500
Cut at xgb probability 0.97
Number of background events after cut: 108009

Model: 5 MeV + 4.6 GeV
Cut at xgb probability 0.94
Number of background events after cut: 278593

Model: 3.0 GeV + 4.6 GeV
Cut at xgb probability 0.95
Number of background events after cut: 277654

Model: all masses
Cut at xgb probability 0.95
Number of background events after cut: 282140



'\n# signal efficiency\nplt.figure(figsize=(9,6))\nplt.plot(bin_edges, 100*(sig_eff[0]), label = legends[0], alpha = 0.7)\nplt.scatter(maximum[0][0], 100*maximum[0][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[1]), label = legends[1], alpha = 0.7)\nplt.scatter(maximum[1][0], 100*maximum[1][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[2]), label = legends[2], alpha = 0.7)\nplt.scatter(maximum[2][0], 100*maximum[2][2], marker = "x")\nplt.plot(bin_edges, 100*(sig_eff[3]), label = legends[3], alpha = 0.7)\nplt.scatter(maximum[3][0], 100*maximum[3][2], marker = "x")\nplt.title("data sample = 4.6 GeV",fontsize=15)\nplt.ylabel("signal efficiency [%]",fontsize=15)\nplt.xlabel("xgb probability", fontsize = 20)\nplt.legend(framealpha = 0)\nplt.savefig(f"/work/aavocone/week12/4/signal4.pdf", format="pdf",bbox_inches="tight")\nplt.show()\n\n#background rejection\nplt.figure(figsize=(9,6))\nplt.plot(bin_edges, 100*(1-bg_eff[0]), label = legends[0], alpha = 0.7)\nplt.scatter(maximum[0]

In [13]:
print("          Model          :      [xgb, max punzi] ")
print("-------------------------------------------------------------")
for i in range(len(labels)):
    print("{0:25s}: {1:}".format(labels[i],maximum[i]))

          Model          :      [xgb, max punzi] 
-------------------------------------------------------------
4.6 GeV with n = 500     : [0.97, 0.000755589897120922, 0.03647, 0.9976467765754575]
(0.005 + 4.6) GeV        : [0.94, 0.00046449423211832735, 0.04608, 0.9939221637265415]
(3.0 + 4.6) GeV          : [0.95, 0.00030882769193584886, 0.04005, 0.9939588500775536]
(0.005 + 3.0 + 4.6) GeV  : [0.94, 0.00025628181744357504, 0.04066, 0.9922133601445238]
